In [1]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from dotenv import load_dotenv
from scipy.stats import pearsonr, spearmanr
from sqlalchemy import create_engine

import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
import random
from deap import base, creator, tools, algorithms

In [2]:
# Load .env from the current directory or specified path
dotenv_path = Path("/Users/pourya/IdeaProjects/DiffBenchmark/.env")
if load_dotenv(dotenv_path):
    print(f"✅ Loaded environment variables from {dotenv_path}")
else:
    print(f"⚠️ Warning: Failed to load .env from {dotenv_path}")

    # Ensure required environment variables exist
required_vars = ["DB_NAME", "DB_USERNAME", "DB_PASSWORD"]
missing = [var for var in required_vars if not os.getenv(var)]
if missing:
    raise ValueError(f"Missing required environment variables: {missing}")

# Database connection settings from environment variables
db_config = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USERNAME'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': os.getenv('DB_PORT', '5432')
}

# SQLAlchemy connection string
conn_str = f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}"
engine = create_engine(conn_str)

# Table name
data_table_name = 'intel'
info_table_name = 'info'

✅ Loaded environment variables from /Users/pourya/IdeaProjects/DiffBenchmark/.env


In [3]:
try:
    df = pd.read_sql(f'SELECT * FROM {data_table_name}', engine)
    infos = pd.read_sql(f'Select * from {info_table_name}', engine)
    #rename the column file to srcPath in infos table
    infos = infos.rename(columns={'file': 'srcpath'})
except Exception as e:
    print(f"❌ Error: {e}")

In [4]:
df[['precision', 'recall', 'f1']] = df[['precision', 'recall', 'f1']].replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=['precision', 'recall', 'f1'])

In [5]:
df

,id,commit,conf,f1,fn_mappings,fn_programelements,fp_mappings,fp_programelements,matcher,n_sum,...,repo,s,srcpath,tp_mappings,tp_programelements,tp_raw_mappings,tp_raw_programelements,trv_mappings,trv_programelements,dataset
0,1,d4992887291cc0a7eeda87ad547fa9e1e7fda41c,"{st_priocalc=size, st_minprio=4}",0.779661,6,1,5,1,SimpleGumtree,51,...,https://github.com/deeplearning4j/deeplearning...,6,deeplearning4j-core/src/main/java/org/deeplear...,21,2,237,28,216,26,None
1,2,d4992887291cc0a7eeda87ad547fa9e1e7fda41c,"{st_priocalc=size, st_minprio=1}",0.800000,5,1,5,1,SimpleGumtree,50,...,https://github.com/deeplearning4j/deeplearning...,8,deeplearning4j-core/src/main/java/org/deeplear...,22,2,238,28,216,26,None
2,3,d4992887291cc0a7eeda87ad547fa9e1e7fda41c,"{st_priocalc=height, st_minprio=3}",0.615385,12,2,5,1,SimpleGumtree,73,...,https://github.com/deeplearning4j/deeplearning...,7,deeplearning4j-core/src/main/java/org/deeplear...,15,1,231,27,216,26,None
3,4,d4992887291cc0a7eeda87ad547fa9e1e7fda41c,"{st_priocalc=height, st_minprio=2}",0.779661,6,1,5,1,SimpleGumtree,51,...,https://github.com/deeplearning4j/deeplearning...,6,deeplearning4j-core/src/main/java/org/deeplear...,21,2,237,28,216,26,None
4,5,d4992887291cc0a7eeda87ad547fa9e1e7fda41c,"{st_priocalc=size, st_minprio=3}",0.779661,6,1,5,1,SimpleGumtree,51,...,https://github.com/deeplearning4j/deeplearning...,6,deeplearning4j-core/src/main/java/org/deeplear...,21,2,237,28,216,26,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629845,629846,7a1659c12d76b510809dea1dea1f5100bcf4cd60,"{st_priocalc=size, st_minprio=3, bu_minsize=2000}",0.964286,4,0,2,0,HybridGumtree,80,...,https://github.com/apache/pig.git,28,src/org/apache/pig/backend/hadoop/executioneng...,77,4,469,29,392,25,None
629846,629847,7a1659c12d76b510809dea1dea1f5100bcf4cd60,"{st_priocalc=size, st_minprio=4, bu_minsize=2000}",0.964286,4,0,2,0,HybridGumtree,80,...,https://github.com/apache/pig.git,28,src/org/apache/pig/backend/hadoop/executioneng...,77,4,469,29,392,25,None
629847,629848,7a1659c12d76b510809dea1dea1f5100bcf4cd60,"{st_priocalc=height, st_minprio=4, bu_minsize=...",0.946108,6,0,3,0,HybridGumtree,88,...,https://github.com/apache/pig.git,22,src/org/apache/pig/backend/hadoop/executioneng...,75,4,467,29,392,25,None
629848,629849,7a1659c12d76b510809dea1dea1f5100bcf4cd60,"{st_priocalc=size, st_minprio=5, bu_minsize=2000}",0.964286,4,0,2,0,HybridGumtree,80,...,https://github.com/apache/pig.git,28,src/org/apache/pig/backend/hadoop/executioneng...,77,4,469,29,392,25,None


In [5]:
def calculate_wess(weights, df):
    w1, w2, w3, w4 = weights
    return w1 * df['ni'] + w2 * df['nd'] + w3 * df['nm'] + w4 * df['nu']

# Normalize WESS per (commit, srcpath)
def normalize_wess_add_score(df, wess, edSizeColumnName):
    df = df.copy()
    df[edSizeColumnName] = wess

    grouped = df.groupby(['commit', 'srcpath'])[edSizeColumnName].agg(['min', 'max']).reset_index()
    grouped.columns = ['commit', 'srcpath', 'min_wess', 'max_wess']
    df = df.merge(grouped, on=['commit', 'srcpath'], how='left')

    df['score'] = (df['max_wess'] - df[edSizeColumnName]) / (df['max_wess'] - df['min_wess'])
    df['score'] = df['score'].replace([np.inf, -np.inf], np.nan)
    df = df.dropna(subset=['score'])

    return df

def corr_calc(w_ni, w_nd, w_nm, w_nu,  metric, df):
    wess = calculate_wess([w_ni, w_nd, w_nm, w_nu], df)
    df_scored = normalize_wess_add_score(df, wess, 'WESS')
    if df_scored.empty:
        return 0
    corr, _ = pearsonr(df_scored['score'], df_scored[metric])
    return corr


In [6]:
target_correl = 1
target_metric = 'precision'

def corr_calc_err(w_ni, w_nd, w_nm, w_nu, metric, df):
    return (corr_calc(w_ni, w_nd, w_nm, w_nu, metric, df) - target_correl ) ** 2

# Genetic Algorithm setup
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # We're minimizing
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Boundaries for ni, nd, nm, nu (adjust as needed)
BOUNDS = [(0, 100), (0, 100), (0, 100), (0, 100)]

# Attribute generator: random float in each variable's range
for i, (low, up) in enumerate(BOUNDS):
    toolbox.register(f"attr_float_{i}", random.uniform, low, up)

# Structure initializers
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_float_0, toolbox.attr_float_1,
                  toolbox.attr_float_2, toolbox.attr_float_3), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Evaluation function
def eval_function(individual):
    w_ni, w_nd, w_nm, w_nu = individual
    return (corr_calc_err(w_ni, w_nd, w_nm, w_nu,target_metric, df),)  # Tuple required

toolbox.register("evaluate", eval_function)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Run GA
def run_ga():
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", lambda x: round(sum(val[0] for val in x) / len(x), 4))
    stats.register("min", lambda x: round(min(val[0] for val in x), 4))

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=100, stats=stats, halloffame=hof, verbose=True)

    print("\nBest solution:", hof[0])
    print("Best fitness:", hof[0].fitness.values[0])

run_ga()


gen	nevals	avg   	min   
0  	300   	0.8549	0.6555
1  	166   	0.8002	0.6555
2  	175   	0.7581	0.6538
3  	170   	0.7255	0.6489
4  	181   	0.6994	0.6394
5  	184   	0.6847	0.491 
6  	199   	0.6797	0.5548
7  	163   	0.6752	0.5723
8  	161   	0.657 	0.4923
9  	178   	0.6476	0.4923
10 	174   	0.6341	0.4905
11 	188   	0.6014	0.4905
12 	157   	0.561 	0.4883
13 	172   	0.532 	0.4784
14 	173   	0.5147	0.4784
15 	198   	0.5056	0.4784
16 	182   	0.5034	0.4765
17 	177   	0.5011	0.4765
18 	204   	0.504 	0.4765
19 	165   	0.4964	0.4763
20 	159   	0.4888	0.4762
21 	186   	0.4871	0.4762
22 	169   	0.4857	0.4762
23 	172   	0.4853	0.4761
24 	180   	0.4844	0.476 
25 	182   	0.484 	0.4759
26 	185   	0.4845	0.4759


KeyboardInterrupt: 

In [9]:
import numpy as np
import pandas as pd
from scipy.optimize import differential_evolution
from scipy.stats import pearsonr

# --- Your functions (unchanged) ---
def calculate_wess(weights, df):
    w1, w2, w3, w4 = weights
    return w1 * df['ni'] + w2 * df['nd'] + w3 * df['nm'] + w4 * df['nu']

def normalize_wess_add_score(df, wess, edSizeColumnName):
    df = df.copy()
    df[edSizeColumnName] = wess

    grouped = df.groupby(['commit', 'srcpath'])[edSizeColumnName].agg(['min', 'max']).reset_index()
    grouped.columns = ['commit', 'srcpath', 'min_wess', 'max_wess']
    df = df.merge(grouped, on=['commit', 'srcpath'], how='left')

    df['score'] = (df['max_wess'] - df[edSizeColumnName]) / (df['max_wess'] - df['min_wess'])
    df['score'] = df['score'].replace([np.inf, -np.inf], np.nan)
    df = df.dropna(subset=['score'])

    return df

def corr_calc(w_ni, w_nd, w_nm, w_nu, metric, df):
    wess = calculate_wess([w_ni, w_nd, w_nm, w_nu], df)
    df_scored = normalize_wess_add_score(df, wess, 'WESS')
    if df_scored.empty:
        return 0
    corr, _ = pearsonr(df_scored['score'], df_scored[metric])
    return corr

# --- Optimization with logging ---
def objective(weights, metric, df):
    corr = corr_calc(*weights, metric, df)
    print(f"Trying weights {weights} -> correlation: {corr:.5f}")
    return -corr

def callback(xk, convergence):
    print(f"Current best weights: {xk} | Convergence: {convergence:.6f}")

bounds = [(0, 1), (0, 1), (0, 1), (0, 1)]

def optimize_weights(df, metric='precision'):
    print("Starting optimization...\n")
    result = differential_evolution(
        objective,
        bounds,
        args=(metric, df),
        callback=callback,
        disp=True,
        seed=42
    )
    best_weights = result.x
    best_corr = -result.fun
    print("\nOptimization complete!")
    print(f"Best weights: {best_weights}")
    print(f"Best correlation: {best_corr:.5f}")
    return best_weights, best_corr


best_weights, best_corr = optimize_weights(df, metric='precision')
print(f"Best Weights: {best_weights}")
print(f"Best Correlation: {best_corr}")


Starting optimization...

Trying weights [0.98801287 0.15162787 0.08820241 0.93988496] -> correlation: 0.00333
Trying weights [0.23480821 0.51493485 0.53030025 0.53703513] -> correlation: 0.09272
Trying weights [0.07173737 0.66269366 0.1033279  0.71957277] -> correlation: 0.03032
Trying weights [0.63574825 0.04513454 0.44017305 0.88728748] -> correlation: 0.10462
Trying weights [0.92762688 0.29978145 0.1494272  0.60837798] -> correlation: 0.01293
Trying weights [0.23232498 0.17491962 0.68867967 0.36062596] -> correlation: 0.13712
Trying weights [0.90542333 0.97487529 0.12679241 0.91415372] -> correlation: -0.00857
Trying weights [0.97628173 0.7204416  0.89520299 0.31215012] -> correlation: 0.08773
Trying weights [0.61752465 0.11308627 0.91822791 0.45268702] -> correlation: 0.12142
Trying weights [0.79216084 0.48013534 0.91082721 0.95837728] -> correlation: 0.09327
Trying weights [0.72520155 0.8693468  0.16723981 0.08943936] -> correlation: -0.00044
Trying weights [0.06387404 0.75189123